### N-gram language models or how to write scientific papers (4 pts)

We shall train our language model on a corpora of [ArXiv](http://arxiv.org/) articles and see if we can generate a new one!

![img](https://media.npr.org/assets/img/2013/12/10/istock-18586699-monkey-computer_brick-16e5064d3378a14e0e4c2da08857efe03c04695e-s800-c85.jpg)

_data by neelshah18 from [here](https://www.kaggle.com/neelshah18/arxivdataset/)_

_Disclaimer: this has nothing to do with actual science. But it's fun, so who cares?!_

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Alternative manual download link: https://yadi.sk/d/_nGyU2IajjR9-w
# !wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
# !tar -xvzf arxivData.json.tar.gz
data = pd.read_json("./arxivData.json")
data.sample(n=5)

,author,day,id,link,month,summary,tag,title,year
26557,"[{'name': 'Christoph Feichtenhofer'}, {'name':...",7,1611.02155v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",11,Two-stream Convolutional Networks (ConvNets) h...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Spatiotemporal Residual Networks for Video Act...,2016
26668,"[{'name': 'Baburaj M.'}, {'name': 'Sudhish N. ...",18,1611.05964v4,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",11,This paper focus on recovering multi-dimension...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Reweighted Low-Rank Tensor Completion and its ...,2016
24006,"[{'name': 'Wim Abbeloos'}, {'name': 'Toon Goed...",7,1612.02223v1,"[{'rel': 'related', 'href': 'http://dx.doi.org...",12,"Combining new, low-cost thermal infrared and t...","[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Exploring the potential of combining time of f...,2016
24725,"[{'name': 'Kriti Kumar'}, {'name': 'Ashley Var...",10,1411.2335v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",11,Mobile Augmented Reality (MAR) is becoming an ...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",An Improved Tracking using IMU and Vision Fusi...,2014
16388,"[{'name': 'Daniele Falavigna'}, {'name': 'Marc...",6,1702.01714v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,In this paper we propose to exploit the automa...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",DNN adaptation by automatic quality estimation...,2017


In [4]:
# assemble lines: concatenate title and description
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'], axis=1).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

### Tokenization

You know the dril. The data is messy. Go clean the data. Use WordPunctTokenizer or something.


In [9]:
# Task: convert lines (in-place) into strings of space-separated tokens. import & use WordPunctTokenizer

# <YOUR CODE>
import nltk
tokenizer = nltk.tokenize.WordPunctTokenizer()

lines = [' '.join(tokenizer.tokenize(line.lower())) for line in lines]

In [11]:
assert sorted(lines, key=len)[0] == \
    'differential contrastive divergence ; this paper has been retracted .'
assert sorted(lines, key=len)[2] == \
    'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .'

### N-Gram Language Model

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$ 

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words. 

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

The first stage to building such a model is counting all word occurences given N-1 previous words

In [187]:
from tqdm import tqdm
from collections import defaultdict, Counter

# special tokens: 
# - unk represents absent tokens, 
# - eos is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"
from time import sleep

def count_ngrams(lines, n):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    When building counts, please consider the following two edge cases
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)
    # counts[(word1, word2)][word3] = how many times word3 occured after (word1, word2)
    for line in tqdm(lines):
        tmp = line.split()
        tmp.append(EOS)
        w_list = [UNK] * (n - 1)
        w_list.extend(tmp)
        
        for i in range(len(w_list) - n + 1):
            key = tuple(w_list[i: i + n - 1])
            next_word = w_list[i + n - 1]
            counts[key].update([next_word])
    return counts


In [188]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)

assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[('_UNK_', '_UNK_')]) == 78
assert dummy_counts['_UNK_', 'a']['note'] == 3
assert dummy_counts['p', '=']['np'] == 2
assert dummy_counts['author', '.']['_EOS_'] == 1



  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:00<00:00, 8230.26it/s]

Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [189]:
class NGramLanguageModel:    
    def __init__(self, lines, n):
        """ 
        Train a simple count-based language model: 
        compute probabilities P(w_t | prefix) given ngram counts
        
        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n
    
        counts = count_ngrams(lines, self.n)
        
        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)
        
        # populate self.probs with actual probabilities
        # <YOUR CODE>
        for k in counts:
            total = sum(counts[k].values())  
            tmp = dict(counts[k])
            tmp = {k: v / total for k, v in tmp.items()}
            self.probs[k] = Counter(tmp)
            
    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]
    
    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

Let's test it!

In [190]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

p_initial = dummy_lm.get_possible_next_tokens('') # '' -> ['_UNK_', '_UNK_']
assert np.allclose(p_initial['learning'], 0.02)
assert np.allclose(p_initial['a'], 0.13)
assert np.allclose(p_initial.get('meow', 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens('a') # '' -> ['_UNK_', 'a']
assert np.allclose(p_a['machine'], 0.15384615)
assert np.allclose(p_a['note'], 0.23076923)
assert np.allclose(p_a.get('the', 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens('a note')['on'], 1)
assert dummy_lm.get_possible_next_tokens('a machine') == \
    dummy_lm.get_possible_next_tokens("there have always been ghosts in a machine"), \
    "your 3-gram model should only depend on 2 previous words"



  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:00<00:00, 8325.83it/s]

Now that you've got a working n-gram language model, let's see what sequences it can generate. But first, let's train it on the whole dataset.

In [191]:
lm = NGramLanguageModel(lines, n=3)



  0%|          | 0/41000 [00:00<?, ?it/s]

  0%|          | 164/41000 [00:00<00:25, 1632.26it/s]

  1%|          | 319/41000 [00:00<00:25, 1605.01it/s]

  1%|          | 460/41000 [00:00<00:26, 1539.48it/s]

  2%|▏         | 627/41000 [00:00<00:25, 1574.18it/s]

  2%|▏         | 784/41000 [00:00<00:25, 1572.58it/s]

  2%|▏         | 942/41000 [00:00<00:25, 1573.04it/s]

  3%|▎         | 1099/41000 [00:00<00:25, 1571.58it/s]

  3%|▎         | 1254/41000 [00:00<00:25, 1563.99it/s]

  3%|▎         | 1409/41000 [00:00<00:25, 1557.49it/s]

  4%|▍         | 1563/41000 [00:01<00:25, 1551.83it/s]

  4%|▍         | 1733/41000 [00:01<00:24, 1592.35it/s]

  5%|▍         | 1891/41000 [00:01<00:24, 1588.49it/s]

  5%|▍         | 2048/41000 [00:01<00:25, 1513.80it/s]

  5%|▌         | 2199/41000 [00:01<00:25, 1512.17it/s]

  6%|▌         | 2359/41000 [00:01<00:25, 1535.55it/s]

  6%|▌         | 2519/41000 [00:01<00:24, 1552.43it/s]

  7%|▋         | 2678/41000 [00:01<00:24, 1562.25it/s]

  7%|▋   

 53%|█████▎    | 21782/41000 [00:14<00:11, 1700.04it/s]

 54%|█████▎    | 21954/41000 [00:14<00:11, 1678.59it/s]

 54%|█████▍    | 22124/41000 [00:14<00:11, 1679.18it/s]

 54%|█████▍    | 22303/41000 [00:14<00:10, 1709.08it/s]

 55%|█████▍    | 22477/41000 [00:14<00:10, 1716.87it/s]

 55%|█████▌    | 22650/41000 [00:14<00:10, 1718.09it/s]

 56%|█████▌    | 22823/41000 [00:14<00:10, 1706.76it/s]

 56%|█████▌    | 22994/41000 [00:14<00:10, 1698.88it/s]

 57%|█████▋    | 23187/41000 [00:14<00:10, 1760.62it/s]

 57%|█████▋    | 23371/41000 [00:15<00:09, 1782.23it/s]

 57%|█████▋    | 23556/41000 [00:15<00:09, 1800.88it/s]

 58%|█████▊    | 23737/41000 [00:15<00:09, 1791.30it/s]

 58%|█████▊    | 23917/41000 [00:15<00:09, 1763.51it/s]

 59%|█████▉    | 24094/41000 [00:15<00:09, 1691.78it/s]

 59%|█████▉    | 24265/41000 [00:15<00:10, 1663.97it/s]

 60%|█████▉    | 24438/41000 [00:15<00:09, 1681.29it/s]

 60%|██████    | 24607/41000 [00:15<00:09, 1659.34it/s]

 60%|██████    | 24774/41000 [0

The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature. In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [192]:
def get_next_token(lm, prefix, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ temperature
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    # <YOUR CODE>
    counter = lm.get_possible_next_tokens(prefix)
    next_words = list(counter)
    probs = list(counter.values())
    if temperature == 0:
        next_word = next_words[probs.index(max(probs))]
    else:
        probs = [x ** (1.0 / temperature) for x in probs]
        sum_prob = sum(probs)
        probs = [x / sum_prob for x in probs]
        next_word = np.random.choice(next_words, p=probs)
    
    return next_word

In [193]:
from collections import Counter
test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])
assert 250 < test_freqs['not'] < 450
assert 8500 < test_freqs['been'] < 9500
assert 1 < test_freqs['lately'] < 200

test_freqs = Counter([get_next_token(lm, 'deep', temperature=1.0) for _ in range(10000)])
assert 1500 < test_freqs['learning'] < 3000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.5) for _ in range(10000)])
assert 8000 < test_freqs['learning'] < 9000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.0) for _ in range(10000)])
assert test_freqs['learning'] == 10000

print("Looks nice!")

Looks nice!


Let's have fun with this model

In [194]:
prefix = 'artificial' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

artificial neural network models which can then be used on a small lower bound for the more challenging domains like network structure gets remodeled automatically by matching the question whether similar q - learning approach to large colonic deformations induced by the model computationally feasible model for the benign of mammogram datasets , mnist - dvs ( dynamic vision sensor networks . we find that our dnn achieves state - of - the - art in the 2012 nobel laureate in economics , business and scientific articles in order to obtain embeddings for other semantic properties of the training process .


In [201]:
prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

bridging the gap between the data . _EOS_


### Evaluating language models: perplexity

Perplexity is a measure of how well does your model approximate true probability distribution behind data. __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences to the power of 1, divided by __total length of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent.

In [202]:
def perplexity(lm, lines, min_logprob=np.log(10 ** -50.)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above
    
    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)
    
    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """
    # <YOUR CODE>        
    ppl = 0
    total = 0
    for i, line in enumerate(lines):
        line = line.split()
        line = [UNK] * (lm.n - 1) + line
        line = line + [EOS]
        
        for i in range(lm.n - 1, len(line)):
            prefix = ' '.join(line[i-lm.n+1: i])
            next_token = line[i]
            prob = lm.get_next_token_prob(prefix=prefix, next_token=next_token)
            log_prob = np.log(prob)
            if log_prob < min_logprob:
                log_prob = min_logprob
            ppl += log_prob
            total += 1
    ppl = np.exp((-1.0) / total * ppl)  
    
    return ppl

In [209]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
lm3 = NGramLanguageModel(dummy_lines, n=3)
lm10 = NGramLanguageModel(dummy_lines, n=10)

ppx1 = perplexity(lm1, dummy_lines)
ppx3 = perplexity(lm3, dummy_lines)
ppx10 = perplexity(lm10, dummy_lines)
ppx_missing = perplexity(lm3, ['the jabberwock , with eyes of flame , '])  # thanks, L. Carrol

print("Perplexities: ppx1=%.3f ppx3=%.3f ppx10=%.3f" % (ppx1, ppx3, ppx10))

assert all(0 < ppx < 500 for ppx in (ppx1, ppx3, ppx10)), "perplexity should be nonnegative and reasonably small"
assert ppx1 > ppx3 > ppx10, "higher N models should overfit and "
assert np.isfinite(ppx_missing) and ppx_missing > 10 ** 6, "missing words should have large but finite perplexity. " \
    " Make sure you use min_logprob right"
assert np.allclose([ppx1, ppx3, ppx10], (318.2132342216302, 1.5199996213739575, 1.1838145037901249))



  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:00<00:00, 13716.29it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:00<00:00, 8700.61it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:00<00:00, 7851.27it/s]

Perplexities: ppx1=318.213 ppx3=1.520 ppx10=1.184


/home/fanshuai/1T/virtual-env/nlp_course/lib/python3.6/site-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in log


Now let's measure the actual perplexity: we'll split the data into train and test and score model on test data only.

In [210]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(lines, test_size=0.25, random_state=42)

for n in (1, 2, 3):
    lm = NGramLanguageModel(n=n, lines=train_lines)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))




  0%|          | 0/30750 [00:00<?, ?it/s]

  1%|          | 242/30750 [00:00<00:12, 2416.67it/s]

  2%|▏         | 482/30750 [00:00<00:12, 2411.35it/s]

  2%|▏         | 725/30750 [00:00<00:12, 2415.35it/s]

  3%|▎         | 966/30750 [00:00<00:12, 2412.52it/s]

  4%|▍         | 1203/30750 [00:00<00:12, 2399.16it/s]

  5%|▍         | 1433/30750 [00:00<00:12, 2368.05it/s]

  5%|▌         | 1674/30750 [00:00<00:12, 2377.99it/s]

  6%|▌         | 1907/30750 [00:00<00:12, 2361.11it/s]

  7%|▋         | 2137/30750 [00:00<00:12, 2341.91it/s]

  8%|▊         | 2374/30750 [00:01<00:12, 2348.88it/s]

  8%|▊         | 2610/30750 [00:01<00:11, 2349.88it/s]

  9%|▉         | 2841/30750 [00:01<00:12, 2323.31it/s]

 10%|█         | 3082/30750 [00:01<00:11, 2346.91it/s]

 11%|█         | 3315/30750 [00:01<00:11, 2341.35it/s]

 12%|█▏        | 3560/30750 [00:01<00:11, 2371.28it/s]

 12%|█▏        | 3801/30750 [00:01<00:11, 2381.89it/s]

 13%|█▎        | 4039/30750 [00:01<00:11, 2350.55it/s]

 14%|█▍

100%|█████████▉| 30672/30750 [00:13<00:00, 2302.06it/s]

100%|██████████| 30750/30750 [00:13<00:00, 2324.43it/s]/home/fanshuai/1T/virtual-env/nlp_course/lib/python3.6/site-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in log


  0%|          | 0/30750 [00:00<?, ?it/s]

  1%|          | 195/30750 [00:00<00:15, 1946.68it/s]

N = 1, Perplexity = 1832.23136




  1%|▏         | 412/30750 [00:00<00:15, 2006.98it/s]

  2%|▏         | 620/30750 [00:00<00:14, 2028.24it/s]

  3%|▎         | 838/30750 [00:00<00:14, 2069.56it/s]

  3%|▎         | 1043/30750 [00:00<00:14, 2060.03it/s]

  4%|▍         | 1248/30750 [00:00<00:14, 2054.81it/s]

  5%|▍         | 1449/30750 [00:00<00:14, 2039.31it/s]

  5%|▌         | 1656/30750 [00:00<00:14, 2048.00it/s]

  6%|▌         | 1852/30750 [00:00<00:14, 2018.72it/s]

  7%|▋         | 2050/30750 [00:01<00:14, 2003.02it/s]

  7%|▋         | 2256/30750 [00:01<00:14, 2017.57it/s]

  8%|▊         | 2462/30750 [00:01<00:13, 2030.04it/s]

  9%|▊         | 2663/30750 [00:01<00:13, 2012.72it/s]

  9%|▉         | 2868/30750 [00:01<00:13, 2023.00it/s]

 10%|▉         | 3074/30750 [00:01<00:13, 2032.09it/s]

 11%|█         | 3277/30750 [00:01<00:13, 2015.27it/s]

 11%|█▏        | 3485/30750 [00:01<00:13, 2033.77it/s]

 12%|█▏        | 3699/30750 [00:01<00:13, 2062.39it/s]

 13%|█▎        | 3906/30750 [00:01<00:13, 2057.35

 86%|████████▌ | 26495/30750 [00:13<00:02, 1899.31it/s]

 87%|████████▋ | 26686/30750 [00:13<00:02, 1899.57it/s]

 87%|████████▋ | 26877/30750 [00:13<00:02, 1869.79it/s]

 88%|████████▊ | 27068/30750 [00:13<00:01, 1881.40it/s]

 89%|████████▊ | 27272/30750 [00:13<00:01, 1925.44it/s]

 89%|████████▉ | 27478/30750 [00:13<00:01, 1962.25it/s]

 90%|█████████ | 27675/30750 [00:14<00:01, 1943.88it/s]

 91%|█████████ | 27870/30750 [00:14<00:01, 1938.04it/s]

 91%|█████████▏| 28076/30750 [00:14<00:01, 1972.85it/s]

 92%|█████████▏| 28274/30750 [00:14<00:01, 1941.92it/s]

 93%|█████████▎| 28469/30750 [00:14<00:01, 1894.87it/s]

 93%|█████████▎| 28659/30750 [00:14<00:01, 1891.43it/s]

 94%|█████████▍| 28849/30750 [00:14<00:01, 1890.54it/s]

 94%|█████████▍| 29039/30750 [00:14<00:00, 1888.11it/s]

 95%|█████████▌| 29228/30750 [00:14<00:00, 1885.71it/s]

 96%|█████████▌| 29417/30750 [00:14<00:00, 1883.46it/s]

 96%|█████████▋| 29606/30750 [00:15<00:00, 1876.23it/s]

 97%|█████████▋| 29796/30750 [0

N = 2, Perplexity = 85653987.28774


  1%|          | 208/30750 [00:00<00:33, 905.54it/s] 

  1%|          | 364/30750 [00:00<00:29, 1035.84it/s]

  2%|▏         | 518/30750 [00:00<00:26, 1148.47it/s]

  2%|▏         | 676/30750 [00:00<00:24, 1250.04it/s]

  3%|▎         | 838/30750 [00:00<00:22, 1340.72it/s]

  3%|▎         | 986/30750 [00:00<00:26, 1125.81it/s]

  4%|▎         | 1143/30750 [00:00<00:24, 1228.65it/s]

  4%|▍         | 1298/30750 [00:00<00:22, 1309.56it/s]

  5%|▍         | 1454/30750 [00:01<00:21, 1375.68it/s]

  5%|▌         | 1608/30750 [00:01<00:20, 1420.20it/s]

  6%|▌         | 1774/30750 [00:01<00:19, 1484.43it/s]

  6%|▋         | 1931/30750 [00:01<00:19, 1508.47it/s]

  7%|▋         | 2097/30750 [00:01<00:18, 1548.75it/s]

  7%|▋         | 2256/30750 [00:01<00:18, 1558.22it/s]

  8%|▊         | 2414/30750 [00:01<00:23, 1225.70it/s]

  8%|▊         | 2570/30750 [00:01<00:21, 1309.37it/s]

  9%|▉         | 2724/30750 [00:01<00:20, 1370.58it/s]

  9%|▉         | 2882/30750 [00:02<00:19, 1426.72it/s]

 69%|██████▊   | 21103/30750 [00:14<00:06, 1600.34it/s]

 69%|██████▉   | 21264/30750 [00:14<00:05, 1583.92it/s]

 70%|██████▉   | 21423/30750 [00:14<00:05, 1583.18it/s]

 70%|███████   | 21592/30750 [00:14<00:05, 1611.33it/s]

 71%|███████   | 21756/30750 [00:14<00:05, 1618.41it/s]

 71%|███████▏  | 21919/30750 [00:14<00:05, 1614.93it/s]

 72%|███████▏  | 22081/30750 [00:14<00:05, 1594.35it/s]

 72%|███████▏  | 22241/30750 [00:15<00:05, 1578.29it/s]

 73%|███████▎  | 22404/30750 [00:15<00:05, 1592.33it/s]

 73%|███████▎  | 22569/30750 [00:15<00:05, 1605.35it/s]

 74%|███████▍  | 22739/30750 [00:15<00:04, 1630.60it/s]

 74%|███████▍  | 22904/30750 [00:15<00:04, 1635.06it/s]

 75%|███████▌  | 23074/30750 [00:15<00:04, 1653.92it/s]

 76%|███████▌  | 23240/30750 [00:15<00:04, 1640.04it/s]

 76%|███████▌  | 23408/30750 [00:15<00:04, 1651.53it/s]

 77%|███████▋  | 23574/30750 [00:15<00:04, 1557.67it/s]

 77%|███████▋  | 23731/30750 [00:15<00:04, 1506.38it/s]

 78%|███████▊  | 23891/30750 [0

N = 3, Perplexity = 61999196259042902147072.00000


In [ ]:
# whoops, it just blew up :)

### LM Smoothing

The problem with our simple language model is that whenever it encounters an n-gram it has never seen before, it assigns it with the probabilitiy of 0. Every time this happens, perplexity explodes.

To battle this issue, there's a technique called __smoothing__. The core idea is to modify counts in a way that prevents probabilities from getting too low. The simplest algorithm here is Additive smoothing (aka [Lapace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)):

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } $$

If counts for a given prefix are low, additive smoothing will adjust probabilities to a more uniform distrivution. Not that the summation in the denominator goes over _all words in the vocabulary_.

Here's an example code we've implemented for you:

In [205]:
class LaplaceLanguageModel(NGramLanguageModel): 
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        self.probs = defaultdict(Counter)

        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {token: (token_counts[token] + delta) / total_count
                                          for token in token_counts}
    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}
    
    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        

In [206]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"



  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:00<00:00, 13680.50it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:00<00:00, 9643.19it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:00<00:00, 9656.73it/s]

In [208]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))



  0%|          | 0/30750 [00:00<?, ?it/s]

  1%|          | 240/30750 [00:00<00:12, 2391.30it/s]

  2%|▏         | 483/30750 [00:00<00:12, 2401.70it/s]

  2%|▏         | 724/30750 [00:00<00:12, 2403.58it/s]

  3%|▎         | 970/30750 [00:00<00:12, 2419.02it/s]

  4%|▍         | 1217/30750 [00:00<00:12, 2431.56it/s]

  5%|▍         | 1453/30750 [00:00<00:12, 2407.39it/s]

  5%|▌         | 1691/30750 [00:00<00:12, 2395.95it/s]

  6%|▋         | 1922/30750 [00:00<00:12, 2368.68it/s]

  7%|▋         | 2155/30750 [00:00<00:12, 2354.43it/s]

  8%|▊         | 2395/30750 [00:01<00:11, 2366.21it/s]

  9%|▊         | 2633/30750 [00:01<00:11, 2367.50it/s]

  9%|▉         | 2867/30750 [00:01<00:11, 2357.78it/s]

 10%|█         | 3100/30750 [00:01<00:11, 2332.55it/s]

 11%|█         | 3332/30750 [00:01<00:11, 2318.05it/s]

 12%|█▏        | 3575/30750 [00:01<00:11, 2348.47it/s]

 12%|█▏        | 3817/30750 [00:01<00:11, 2368.25it/s]

 13%|█▎        | 4054/30750 [00:01<00:11, 2360.66it/s]

 14%|█▍

100%|█████████▉| 30601/30750 [00:13<00:00, 2275.44it/s]

100%|██████████| 30750/30750 [00:13<00:00, 2311.42it/s]

  0%|          | 0/30750 [00:00<?, ?it/s]

  1%|          | 200/30750 [00:00<00:15, 1991.43it/s]

N = 1, Perplexity = 977.67559




  1%|▏         | 410/30750 [00:00<00:15, 2021.94it/s]

  2%|▏         | 611/30750 [00:00<00:14, 2016.24it/s]

  3%|▎         | 816/30750 [00:00<00:14, 2024.39it/s]

  3%|▎         | 1018/30750 [00:00<00:14, 2019.76it/s]

  4%|▍         | 1230/30750 [00:00<00:14, 2046.29it/s]

  5%|▍         | 1426/30750 [00:00<00:14, 2019.03it/s]

  5%|▌         | 1637/30750 [00:00<00:14, 2043.96it/s]

  6%|▌         | 1840/30750 [00:00<00:14, 2038.32it/s]

  7%|▋         | 2045/30750 [00:01<00:14, 2040.74it/s]

  7%|▋         | 2253/30750 [00:01<00:13, 2050.40it/s]

  8%|▊         | 2464/30750 [00:01<00:13, 2065.87it/s]

  9%|▊         | 2668/30750 [00:01<00:13, 2049.46it/s]

  9%|▉         | 2872/30750 [00:01<00:13, 2031.69it/s]

 10%|█         | 3077/30750 [00:01<00:13, 2035.12it/s]

 11%|█         | 3280/30750 [00:01<00:13, 2025.10it/s]

 11%|█▏        | 3493/30750 [00:01<00:13, 2053.15it/s]

 12%|█▏        | 3704/30750 [00:01<00:13, 2067.89it/s]

 13%|█▎        | 3911/30750 [00:01<00:12, 2065.25

 87%|████████▋ | 26685/30750 [00:13<00:02, 1925.57it/s]

 87%|████████▋ | 26878/30750 [00:13<00:02, 1893.28it/s]

 88%|████████▊ | 27068/30750 [00:13<00:01, 1892.47it/s]

 89%|████████▊ | 27266/30750 [00:13<00:01, 1917.60it/s]

 89%|████████▉ | 27466/30750 [00:13<00:01, 1938.40it/s]

 90%|████████▉ | 27661/30750 [00:13<00:01, 1898.86it/s]

 91%|█████████ | 27858/30750 [00:14<00:01, 1919.26it/s]

 91%|█████████▏| 28063/30750 [00:14<00:01, 1955.59it/s]

 92%|█████████▏| 28261/30750 [00:14<00:01, 1961.41it/s]

 93%|█████████▎| 28458/30750 [00:14<00:01, 1960.80it/s]

 93%|█████████▎| 28659/30750 [00:14<00:01, 1974.16it/s]

 94%|█████████▍| 28857/30750 [00:14<00:00, 1971.65it/s]

 94%|█████████▍| 29055/30750 [00:14<00:00, 1961.27it/s]

 95%|█████████▌| 29252/30750 [00:14<00:00, 1948.91it/s]

 96%|█████████▌| 29457/30750 [00:14<00:00, 1973.89it/s]

 96%|█████████▋| 29659/30750 [00:14<00:00, 1987.43it/s]

 97%|█████████▋| 29859/30750 [00:15<00:00, 1990.17it/s]

 98%|█████████▊| 30074/30750 [0

N = 2, Perplexity = 470.48021


  1%|          | 227/30750 [00:00<00:27, 1116.32it/s]

  1%|▏         | 392/30750 [00:00<00:24, 1236.07it/s]

  2%|▏         | 553/30750 [00:00<00:22, 1328.61it/s]

  2%|▏         | 717/30750 [00:00<00:21, 1406.41it/s]

  3%|▎         | 879/30750 [00:00<00:20, 1463.89it/s]

  3%|▎         | 1016/30750 [00:00<00:26, 1134.16it/s]

  4%|▍         | 1184/30750 [00:00<00:23, 1255.84it/s]

  4%|▍         | 1337/30750 [00:00<00:22, 1326.68it/s]

  5%|▍         | 1493/30750 [00:01<00:21, 1387.56it/s]

  5%|▌         | 1642/30750 [00:01<00:20, 1414.32it/s]

  6%|▌         | 1788/30750 [00:01<00:20, 1427.45it/s]

  6%|▋         | 1934/30750 [00:01<00:24, 1191.35it/s]

  7%|▋         | 2095/30750 [00:01<00:22, 1291.80it/s]

  7%|▋         | 2251/30750 [00:01<00:20, 1361.70it/s]

  8%|▊         | 2408/30750 [00:01<00:20, 1416.28it/s]

  8%|▊         | 2562/30750 [00:01<00:19, 1450.98it/s]

  9%|▉         | 2720/30750 [00:01<00:18, 1486.89it/s]

  9%|▉         | 2881/30750 [00:02<00:18, 1519.78it/s

 70%|██████▉   | 21424/30750 [00:14<00:06, 1334.17it/s]

 70%|███████   | 21591/30750 [00:14<00:06, 1418.96it/s]

 71%|███████   | 21750/30750 [00:14<00:06, 1464.92it/s]

 71%|███████▏  | 21914/30750 [00:14<00:05, 1513.14it/s]

 72%|███████▏  | 22071/30750 [00:14<00:05, 1517.06it/s]

 72%|███████▏  | 22233/30750 [00:14<00:05, 1543.99it/s]

 73%|███████▎  | 22396/30750 [00:14<00:05, 1567.48it/s]

 73%|███████▎  | 22573/30750 [00:15<00:05, 1621.79it/s]

 74%|███████▍  | 22738/30750 [00:15<00:04, 1611.15it/s]

 74%|███████▍  | 22901/30750 [00:15<00:04, 1599.39it/s]

 75%|███████▌  | 23072/30750 [00:15<00:04, 1630.01it/s]

 76%|███████▌  | 23236/30750 [00:15<00:04, 1604.78it/s]

 76%|███████▌  | 23401/30750 [00:15<00:04, 1617.33it/s]

 77%|███████▋  | 23564/30750 [00:15<00:04, 1612.09it/s]

 77%|███████▋  | 23726/30750 [00:15<00:04, 1602.21it/s]

 78%|███████▊  | 23887/30750 [00:15<00:04, 1593.46it/s]

 78%|███████▊  | 24053/30750 [00:15<00:04, 1612.62it/s]

 79%|███████▉  | 24237/30750 [0

N = 3, Perplexity = 3679.44765


In [ ]:
# optional: try to sample tokens from such a model

### Kneser-Ney smoothing

Additive smoothing is simple, reasonably good but definitely not a State of The Art algorithm.


Your final task in this notebook is to implement [Kneser-Ney](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing) smoothing.

It can be computed recurrently, for n>1:

$$P_{kn}(w_t | prefix_{n-1}) = { \max(0, Count(prefix_{n-1}, w_t) - \delta) \over \sum_{\hat w} Count(prefix_{n-1}, \hat w)} + \lambda_{prefix_{n-1}} \cdot P_{kn}(w_t | prefix_{n-2})$$

where
- $prefix_{n-1}$ is a tuple of {n-1} previous tokens
- $lambda_{prefix_{n-1}}$ is a normalization constant chosen so that probabilities add up to 1
- Unigram $P_{kn}(w_t | prefix_{n-2})$ corresponds to Kneser Ney smoothing for {N-1}-gram language model.
- Unigram $P_{kn}(w_t)$ is a special case: how likely it is to see x_t in an unfamiliar context

See lecture slides or wiki for more detailed formulae.

__Your task__ is to
- implement KneserNeyLanguageModel
- test it on 1-3 gram language models
- find optimal (within one order of magnitude) smoothing delta for 3-gram language model with Kneser-Ney smoothing

In [ ]:
class KneserNeyLanguageModel(NGramLanguageModel): 
    """ A template for Kneser-Ney language model. Default delta may be suboptimal. """
    def __init__(self, lines, n, delta=5.0):
        self.n = n
        <YOUR CODE>
        
    def get_possible_next_tokens(self, prefix):
        < YOUR CODE >
        
    def get_next_token_prob(self, prefix, next_token):
        <YOUR CODE>

In [ ]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [ ]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, smoothing=<...>)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))